# Determine threshold for considering precipitation "rain"
Predictions by the model do not appear too sensitive to the exact threshold but rather to keypoints/content of the Tweet to rain classification task. Therefore
- We calculate the f1 depending on this threshold for predictions of the same model
- Determined f1 of minority class is max at f1=0.67 for precipitation_threshold=7e-3 m


In [ ]:
# allows update of external libraries without need to reload package
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import re
import glob
import zipfile
import pathlib
import logging
import json
import string

import numpy as np
import pandas as pd
import matplotlib.font_manager
import matplotlib.pyplot as plt
import matplotlib

import matplotlib.offsetbox


import wget

import requests
import base64
from bs4 import BeautifulSoup

import wordcloud
import tweepy
import a2.twitter.downloader
import a2.plotting
import a2.dataset
import a2.utils
import a2.training
import a2.plotting.analysis

import joblib

In [ ]:
FOLDER_DATA = a2.utils.file_handling.get_folder_data()
FOLDER_TWEETS = FOLDER_DATA / "tweets/"
FILE_TWEETS = (
    FOLDER_TWEETS
    / "2017_2020_tweets_rain_sun_vocab_emojis_locations_bba_Tp_era5_no_bots_normalized_filtered_weather_stations_fix_predicted_simpledeberta_radar.nc"
)
FOLDER_FIGURES = pathlib.Path("../../figures/analysis/modelling/")
FILE_FONT = "../fonts/noto/NotoEmoji-VariableFont_wght.ttf"
FOLDER_MODEL_TO_LOAD = "../../models/model_weights/output_rainprediction_simpledeberta/era5/checkpoint-7617/"
FOLDER_RAW_MODEL = pathlib.Path("../../models/deberta-v3-small/")

In [ ]:
ds_tweets = a2.dataset.load_dataset.load_tweets_dataset(FILE_TWEETS)

In [ ]:
ds_tweets

In [ ]:
ds_p = ds_tweets[["tp_h", "prediction"]]
ds_p["tp_h_in_m"] = (["index"], ds_p["tp_h"].values * 1e3)

In [ ]:
ds_p["tp_h_in_m"].max()

In [ ]:
thresholds = 10 ** np.linspace(-5, -1, 100)
prediction = ds_p["prediction"].values
rain_correct_list = []
not_rain_correct_list = []

f1_raining_list = []
f1_not_raining_list = []
for p_thresh in thresholds:
    rain = ds_p["tp_h_in_m"].values > p_thresh
    report = a2.plotting.analysis.classification_report(rain, prediction)
    f1_not_raining = report["not raining"]["f1-score"]
    f1_raining = report["raining"]["f1-score"]
    f1_not_raining_list.append(f1_not_raining)
    f1_raining_list.append(f1_raining)
    rain_correct = np.zeros_like(prediction)
    not_rain_correct = np.zeros_like(prediction)
    rain_correct[np.logical_and(prediction == rain, rain == 1)] = 1
    not_rain_correct[np.logical_and(prediction == rain, rain == 0)] = 1
    rain_correct_list.append(np.sum(rain_correct))
    not_rain_correct_list.append(np.sum(not_rain_correct))

In [ ]:
fig, ax = plt.subplots()
ax.plot(thresholds, f1_raining_list, label="f1 raining")
ax.plot(thresholds, f1_not_raining_list, label="f1 not raining")
ax.set_xscale("log")
ax.set_yscale("log")
ax.legend()

In [ ]:
fig, ax = plt.subplots()
ax.plot(thresholds, rain_correct_list, label="rain correct")
ax.plot(thresholds, not_rain_correct_list, label="not rain correct")
ax.set_xscale("log")
ax.set_yscale("log")
ax.legend()

In [ ]:
truth = ds_p["tp_h_in_m"].values > 7e-3
predictions = ds_p["prediction"].values
prediction_probabilities = np.array(
    [ds_tweets["prediction_probability_not_raining"].values, ds_tweets["prediction_probability_raining"].values]
).T

report = a2.plotting.analysis.classification_report(truth, predictions)
f1_not_raining = report["not raining"]["f1-score"]
f1_raining = report["raining"]["f1-score"]
f1_not_raining, f1_raining

In [ ]:
report = a2.plotting.analysis.check_prediction(truth, predictions)
a2.plotting.analysis.plot_prediction_certainty(truth=truth, prediction_probabilities=prediction_probabilities[:, 1])
a2.plotting.analysis.plot_roc(truth, prediction_probabilities[:, 1])
print(report)